<a href="https://colab.research.google.com/github/angelaaaateng/ML_COVID_PREDICTION/blob/master/Exploration_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Final Project: COVID Prediction in the Philippines 


Team members: 

Overview: 

Preprocessing Sequence: 
- openrefine clustering/data cleaning (done outside this notebook, see scripts) 
- Symptoms + Other Diseases Feature Engineering 
- Symptops + Other Diseases one hot encoding
- Epi_Link Feature engineering 
- Dates feature engineering
- Geo-location feature engineering (absolute location) 
- Location feature engineering (zip) 
- Location feature engineering (city) 
- Population Featue engineering
- Modeling
- Clustering (Target Variable) 

Naming Convention: 
- Amber: data = main dataframe

## Import Libraries + Load Data

In [55]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [54]:
# amber additions
!pwd

/Users/angelateng/Desktop/Github_Files_Teng/ML_COVID_PREDICTION


**DSP Data (Key = PHCaseNo)**

In [79]:
data = pd.read_csv('openrefine_PHCASES_COVID_04162020_PHnum.csv')
data.head()

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History 1,Travel History 2,Travel History 3,Travel History 4,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Location 1,Location 2,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,31 - 40,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,china,NaN,NaN,NaN,...,Sat Feb 01 00:00:00 EST 2020,severe pneumonia,41 - 50,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
2,PH00003,F,60,chinese,None,Yes (China),china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,51 - 60,Manila City,Manila City,NaN,14.615867,120.980991,14.598727,120.9819909
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),japan,NaN,NaN,NaN,...,Thu Mar 19 00:00:00 EDT 2020,"asymptomatic, negative test result",41 - 50,Taguig City,Taguig City,NaN,14.606904,121.020392,14.517274,121.0496133
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,none,NaN,NaN,NaN,...,Thu Mar 12 00:00:00 EDT 2020,"acute respiratory distress syndrome, severe pn...",61 - 70,"Cainta, Rizal",Cainta,Rizal,14.409635,121.037425,14.586506,121.1138255


In [80]:
data.columns

Index(['Case No.', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Travel History 1', 'Travel History 2',
       'Travel History 3', 'Travel History 4', 'Epi_Link',
       'Date of Onset of Symptoms', 'Date of Admission',
       'Date of Lab Confirmation', 'Date of Announcement to the Public',
       'Admission / Consultation', 'Symptoms', 'Other disease', 'Status',
       'Health Status', 'Date of Final Status (recovered/expired)',
       'Final Diagnosis', 'Age Group', 'Location', 'Location 1', 'Location 2',
       'Latitude', 'Longitude', 'Residence Lat', 'Residence Long'],
      dtype='object')

In [81]:
data = data.rename(columns={'Case No.': 'phcaseno'})

In [82]:
# data.dtypes

In [83]:
data.shape

(3246, 30)

In [84]:
# target variable information (before clustering)
data.groupby('Health Status').size()

Health Status
asymptomatic        38
critical             3
died               108
for validation    2983
mild                57
recovered           51
severe               6
dtype: int64

## Feature Engineering: Symptoms + Other Diseases

## One Hot Encoding: Symptoms + Other Diseases 

In [85]:
data.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History 1,Travel History 2,Travel History 3,Travel History 4,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Location 1,Location 2,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,31 - 40,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,china,NaN,NaN,NaN,...,Sat Feb 01 00:00:00 EST 2020,severe pneumonia,41 - 50,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
2,PH00003,F,60,chinese,None,Yes (China),china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,51 - 60,Manila City,Manila City,NaN,14.615867,120.980991,14.598727,120.9819909
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),japan,NaN,NaN,NaN,...,Thu Mar 19 00:00:00 EDT 2020,"asymptomatic, negative test result",41 - 50,Taguig City,Taguig City,NaN,14.606904,121.020392,14.517274,121.0496133
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,none,NaN,NaN,NaN,...,Thu Mar 12 00:00:00 EDT 2020,"acute respiratory distress syndrome, severe pn...",61 - 70,"Cainta, Rizal",Cainta,Rizal,14.409635,121.037425,14.586506,121.1138255


In [86]:
# drop empty columns
data = data.drop(['Travel History 1',
 'Travel History 2',
 'Travel History 3',
 'Travel History 4'], axis = 1) 
data.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Location 1,Location 2,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,Sun Mar 15 00:00:00 EDT 2020,NaN,31 - 40,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,Sat Feb 01 00:00:00 EST 2020,severe pneumonia,41 - 50,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,Sun Mar 15 00:00:00 EDT 2020,NaN,51 - 60,Manila City,Manila City,NaN,14.615867,120.980991,14.598727,120.9819909
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,Thu Mar 19 00:00:00 EDT 2020,"asymptomatic, negative test result",41 - 50,Taguig City,Taguig City,NaN,14.606904,121.020392,14.517274,121.0496133
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,Thu Mar 12 00:00:00 EDT 2020,"acute respiratory distress syndrome, severe pn...",61 - 70,"Cainta, Rizal",Cainta,Rizal,14.409635,121.037425,14.586506,121.1138255


In [87]:
symptoms = data.set_index('phcaseno').Symptoms.str.split(',', expand=True).stack()

In [88]:
# symptoms 

In [89]:
symptoms= pd.get_dummies(symptoms, prefix='symp').groupby(level=0).sum()
# symptoms

In [90]:
# join it back to the original 
data = data.merge(symptoms, on=['phcaseno'], how='left', indicator=True)

In [91]:
data.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,symp_ headache,symp_ shortness of breath,symp_ sore throat,symp_asymptomatic,symp_cough,symp_difficulty of breathing,symp_dyspnea,symp_fever,symp_sore throat,_merge
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,both
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,both
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,both


In [92]:
cols = ['symp_ body malaise',
       'symp_ colds', 'symp_ comorbidity', 'symp_ cough',
       'symp_ difficulty in breathing', 'symp_ dizziness', 'symp_ dysphagia',
       'symp_ headache', 'symp_ shortness of breath', 'symp_ sore throat',
       'symp_asymptomatic', 'symp_cough', 'symp_difficulty of breathing',
       'symp_dyspnea', 'symp_fever', 'symp_sore throat']

In [93]:
data[cols] = data[cols].replace({np.nan:0.0})

**Table with One Hot Encoded Symptoms** 

In [94]:
data = data.drop(['_merge'], axis =1)

In [95]:
data['Other disease'].values.tolist()
data = data.rename(columns={'Other disease':'other_disease'}, errors="raise")

In [96]:
disease = data.set_index('phcaseno').other_disease.str.split(',', expand=True).stack()
# disease

In [98]:
disease_all= pd.get_dummies(disease, prefix='dis').groupby(level=0).sum()
# disease_all

In [99]:
data = data.merge(disease_all, on=['phcaseno'], how='left', indicator=True)

In [101]:
cols = ['dis_  diabetes',
 'dis_  renal disease',
 'dis_ acute kidney injury',
 'dis_ asthma',
 'dis_ atherosclerosis',
 'dis_ atherosclerotic cardiovascular disease',
 'dis_ benign prostatic hyperplasia',
 'dis_ benign prostatic hypertrophy',
 'dis_ bronchial asthma',
 'dis_ cardiac disease',
 'dis_ cardiac dysrhythmia',
 'dis_ cardiomyopathy',
 'dis_ cardiovascular disease',
 'dis_ cerebrovascular accident infarc',
 'dis_ cerebrovascular disease infarction',
 'dis_ chronic kidney disease',
 'dis_ chronic obstructive pulmonary disease',
 'dis_ colon cancer',
 'dis_ coronary artery disease',
 'dis_ coronary heart disease',
 'dis_ dengue',
 'dis_ diabetes',
 'dis_ diabetes mellitus',
 'dis_ diabetic ketoacidosis',
 'dis_ diabetic nephropathy',
 'dis_ diabetis mellitus',
 'dis_ dyslipidemia',
 'dis_ endocrine condition',
 'dis_ hypertension',
 'dis_ hypertensive cardiovascular disease',
 'dis_ hypertensive nephrosclerosis',
 'dis_ hyperthyroidism',
 'dis_ hypothyroidism',
 'dis_ impaired fasting glucose',
 'dis_ ischemic heart disease',
 'dis_ kidney disease',
 'dis_ peripheral arterial occlusive disease',
 'dis_ renal disease',
 'dis_ tongue cancer',
 'dis_ valvular heart disease',
 'dis_acute kidney injury',
 'dis_acute kidney injury secondary to pre-renal azotemia',
 'dis_acute renal failure',
 'dis_asthma',
 'dis_asymptomatic',
 'dis_breast cancer',
 'dis_bronchial asthma',
 'dis_cardiac disease',
 'dis_chronic kidney disease',
 'dis_chronic kidney disease secondary to hypertension nephrosclerosis',
 'dis_chronic obstructive pulmonary disease',
 'dis_community-acquired pneumonia',
 'dis_diabetes',
 'dis_hypertension',
 'dis_hypertensive',
 'dis_hypertensive cardiovascular disease',
 'dis_hyperthyroidism',
 'dis_myocarditis',
 'dis_none',
 'dis_permanent atrial fibrillation',
 'dis_prostatic cancer',
 'dis_remnal disease',
 'dis_status post renal transplant',
 'dis_systemic lupus erythematosus',
 'dis_with comorbid (not specified)']

In [102]:
data[cols] = data[cols].replace({np.nan:0.0})

In [103]:
data

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,dis_hyperthyroidism,dis_myocarditis,dis_none,dis_permanent atrial fibrillation,dis_prostatic cancer,dis_remnal disease,dis_status post renal transplant,dis_systemic lupus erythematosus,dis_with comorbid (not specified),_merge
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3241,PH03242,F,65,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3242,PH03243,F,24,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3243,PH03244,M,48,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3244,PH03245,M,67,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only


**The `symptoms` and `other_disease` columns have no been feature engineered.**

In [104]:
data = data.drop(['_merge'], axis = 1) 
data.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,dis_hypertensive cardiovascular disease,dis_hyperthyroidism,dis_myocarditis,dis_none,dis_permanent atrial fibrillation,dis_prostatic cancer,dis_remnal disease,dis_status post renal transplant,dis_systemic lupus erythematosus,dis_with comorbid (not specified)
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
data.shape

(3246, 107)

## Feature Engineering: Dates

In [112]:
# data.columns.tolist()

In [115]:
data['Date of Onset of Symptoms'] = pd.to_datetime(data['Date of Onset of Symptoms'])
data['Date of Announcement to the Public'] = pd.to_datetime(data['Date of Announcement to the Public'])
data['Date of Final Status (recovered/expired)'] = pd.to_datetime(data['Date of Final Status (recovered/expired)'])


In [ ]:
# add new column

data['disease_len_days'] = data['Date of Final Status (recovered/expired)'] - data['Date of Onset of Symptoms']

## Feature Engineering: Geo-location (absolute location)

In [ ]:
# note: RUN THIS LAST, it takes forever to run

In [107]:
# %load_ext au totime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
import time

import plotly_express as px

In [108]:
geolocator = Nominatim()
# geolocator = Nominatim(domain='localhost:8888/nominatim', scheme='http')
# geolocator=Nominatim(domain='localhost:8080', scheme='http')

/Users/angelateng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning:

Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.



In [109]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]), timeout = 100)
    time.sleep(1.5)
    print("Currently at index", df['phcaseno'], "/", len(df))
    return location.raw['address']

In [80]:
geolocator = geopy.Nominatim(user_agent='my-application')

In [45]:
geo_address = data.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')

In [84]:
geo_address

0       {'hospital': 'San Lazaro Hospital', 'road': 'Q...
1       {'hospital': 'San Lazaro Hospital', 'road': 'Q...
2       {'public_building': 'Francisco Duque Hall', 'r...
3       {'hospital': 'University of the East - Ramon M...
4       {'hospital': 'Research Institute for Tropical ...
                              ...                        
3241    {'public_building': 'Francisco Duque Hall', 'r...
3242    {'public_building': 'Francisco Duque Hall', 'r...
3243    {'public_building': 'Francisco Duque Hall', 'r...
3244    {'public_building': 'Francisco Duque Hall', 'r...
3245    {'public_building': 'Francisco Duque Hall', 'r...
Length: 3246, dtype: object

In [107]:
len(data)

3246

In [125]:
geo_string 

['Main Building',
 ' Wilson',
 ' Greenhills',
 ' Addition Hills',
 ' 2nd District',
 ' San Juan',
 ' Metro Manila',
 ' 1502',
 ' Philippines']

In [85]:
data['geo_address'] = geo_address

## Getting City Data 

In [41]:
# this is using Lauren's CSV file
data = pd.read_csv('data_clean.csv')
data.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,dis_myocarditis,dis_none,dis_permanent atrial fibrillation,dis_prostatic cancer,dis_remnal disease,dis_status post renal transplant,dis_systemic lupus erythematosus,dis_with comorbid (not specified),geo_address,disease_len_days
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,2020-01-21 00:00:00-05:00,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'hospital': 'San Lazaro Hospital', 'road': 'Q...",53 days 23:00:00.000000000
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,2020-01-18 00:00:00-05:00,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'hospital': 'San Lazaro Hospital', 'road': 'Q...",14 days 00:00:00.000000000
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,2020-01-21 00:00:00-05:00,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"{'public_building': 'Francisco Duque Hall', 'r...",53 days 23:00:00.000000000
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,2020-03-03 00:00:00-05:00,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,{'hospital': 'University of the East - Ramon M...,15 days 23:00:00.000000000
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,2020-02-25 00:00:00-05:00,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,{'hospital': 'Research Institute for Tropical ...,15 days 23:00:00.000000000


In [43]:
data['geo_address'][0]

"{'hospital': 'San Lazaro Hospital', 'road': 'Quiricada Street', 'suburb': 'Barangay 325', 'city': 'Santa Cruz', 'municipality': 'Third District', 'region': 'Metro Manila', 'postcode': '1003', 'country': 'Philippines', 'country_code': 'ph'}"

In [44]:
import ast

In [22]:
type(data['geo_address'][0])

str

In [25]:
tester = ast.literal_eval(data['geo_address'][0])
tester

{'hospital': 'San Lazaro Hospital',
 'road': 'Quiricada Street',
 'suburb': 'Barangay 325',
 'city': 'Santa Cruz',
 'municipality': 'Third District',
 'region': 'Metro Manila',
 'postcode': '1003',
 'country': 'Philippines',
 'country_code': 'ph'}

In [32]:
ast.literal_eval(data['geo_address'][0])['city']

'Santa Cruz'

In [70]:
data['city'] = data['geo_address'].apply(lambda x: ast.literal_eval(x))

In [71]:
data['city'][0]['city']

'Santa Cruz'

In [82]:
print(type(data['city'][0]))
(data['city'][0])

<class 'dict'>


{'hospital': 'San Lazaro Hospital',
 'road': 'Quiricada Street',
 'suburb': 'Barangay 325',
 'city': 'Santa Cruz',
 'municipality': 'Third District',
 'region': 'Metro Manila',
 'postcode': '1003',
 'country': 'Philippines',
 'country_code': 'ph'}

In [84]:
(data['city'][0]).get("city")

'Santa Cruz'

In [5]:
!ls

Analysis_Modeling_COVID19.ipynb
COVID_04162020_PHnum.xlsx
CodaLab Instructions
EDA_COVID19_amber-annotated.ipynb
EDA_COVID19_amber.ipynb
Icon?
PROJECT
README.md
data_clean.csv
data_clean_w_addresses.csv
data_clean_w_zip.csv
data_covidph
feature_engineering_LD.ipynb
open_refine
openrefine_PHCASES_COVID_04162020_PHnum.csv
openrefine_phcases.rtf


In [20]:
df = pd.read_csv('data_clean_w_addresses.csv')

In [21]:
df.head()

,Unnamed: 0,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,...,Epi_Link_Traveled,Epi_Link_Exposure,Admission_dt,SymptomOnset_dt,Final_dt,diff_admit_symp,diff_final_admit,diff_final_symp,geom,address
0,0,PH00001,F,38.0,Chinese,None,Yes (China); Wife of PH02,China,['wife of ph2'],21-Jan-20,...,0,1,2020-01-25,2020-01-21,2020-03-15,4 days 00:00:00.000000000,50 days 00:00:00.000000000,54 days 00:00:00.000000000,"14.61348,120.98095","San Lazaro Hospital, Quiricada Street, Baranga..."
1,1,PH00002,M,44.0,Chinese,None,Yes (China); Husband of PH1,China,['husband of ph1'],18-Jan-20,...,0,1,2020-01-25,2020-01-18,2020-02-01,7 days 00:00:00.000000000,7 days 00:00:00.000000000,14 days 00:00:00.000000000,"14.61348,120.98095","San Lazaro Hospital, Quiricada Street, Baranga..."
2,2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,...,1,0,2020-01-22,2020-01-21,2020-03-15,1 days 00:00:00.000000000,53 days 00:00:00.000000000,54 days 00:00:00.000000000,"14.615867000000001,120.98099099999999","Francisco Duque Hall, Tayuman Street, Santa Cr..."
3,3,PH00004,M,48.0,Filipino,Taguig City,Yes (Japan),Japan,['NaN'],03-Mar-20,...,0,1,2020-03-06,2020-03-03,2020-03-19,3 days 00:00:00.000000000,13 days 00:00:00.000000000,16 days 00:00:00.000000000,"14.6069038,121.02039180000001",University of the East - Ramon Magsaysay Memor...
4,4,PH00005,M,62.0,Filipino,"Cainta, Rizal",Husband of PH06,None,['husband of ph6'],25-Feb-20,...,0,1,2020-03-05,2020-02-25,2020-03-12,9 days 00:00:00.000000000,7 days 00:00:00.000000000,16 days 00:00:00.000000000,"14.409634699999998,121.03742450000001","Research Institute for Tropical Medicine, Corp..."


In [23]:
df['address'][0]

'San Lazaro Hospital, Quiricada Street, Barangay 325, Manila, Third District, NCR, First District, Metro Manila, 1003, Philippines'

In [25]:
df_new = pd.concat([df, df['address'].str.split(', ', expand=True)], axis=1)
df_new

,Unnamed: 0,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,...,2,3,4,5,6,7,8,9,10,11
0,0,PH00001,F,38.0,Chinese,None,Yes (China); Wife of PH02,China,['wife of ph2'],21-Jan-20,...,Barangay 325,Manila,Third District,NCR,First District,Metro Manila,1003,Philippines,None,None
1,1,PH00002,M,44.0,Chinese,None,Yes (China); Husband of PH1,China,['husband of ph1'],18-Jan-20,...,Barangay 325,Manila,Third District,NCR,First District,Metro Manila,1003,Philippines,None,None
2,2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,...,Santa Cruz,Manila,Third District,NCR,First District,Metro Manila,1942,Philippines,None,None
3,3,PH00004,M,48.0,Filipino,Taguig City,Yes (Japan),Japan,['NaN'],03-Mar-20,...,Doña Imelda,Galas,4th District,NCR,Second District,Metro Manila,1016,Philippines,None,None
4,4,PH00005,M,62.0,Filipino,"Cainta, Rizal",Husband of PH06,None,['husband of ph6'],25-Feb-20,...,Alabang,Muntinlupa,NCR,Fourth District,Metro Manila,1781,Philippines,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3013,3013,PH03014,F,31.0,For validation,For validation,NaN,For validation,['NaN'],NaN,...,Santa Cruz,Manila,Third District,NCR,First District,Metro Manila,1942,Philippines,None,None
3014,3014,PH03015,F,20.0,For validation,For validation,NaN,For validation,['NaN'],NaN,...,Santa Cruz,Manila,Third District,NCR,First District,Metro Manila,1942,Philippines,None,None
3015,3015,PH03016,M,68.0,For validation,For validation,NaN,For validation,['NaN'],NaN,...,Santa Cruz,Manila,Third District,NCR,First District,Metro Manila,1942,Philippines,None,None
3016,3016,PH03017,F,39.0,For validation,For validation,NaN,For validation,['NaN'],NaN,...,Santa Cruz,Manila,Third District,NCR,First District,Metro Manila,1942,Philippines,None,None


In [ ]:

# data.to_csv('/Users/angelateng/NYU_GoogleDrive/GitHub/ML_COVID_PREDICTION/data_clean.csv', index= False, header=True)